In [2]:
import sys
import pandas as pd #import csv
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QGridLayout, QFileDialog, QLabel, QStackedWidget
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import uic, QtGui
import math

global type_to # 타입 변경을 위한 변수
global type_combo_res
global data # 비식별 전
global count # 데이터셋 중 수치형 데이터의 개수
global info_header
global preview
preview = 10 # 프리뷰로 보여줄 데이터 개수. 기존의 len(data)
info_header = ['이름', '최솟값', '최댓값', '평균', '중앙값', '표준편차', '최빈값'] # 수치형 info
type_combo_res=[]

In [3]:
# 비식별 기법
class anonymous() :
    def __init__(self,df):
        self.df = df
        self.backup = backup
    
    def delete_col(self, column):
        """
            delete_col : 열 삭제
            column : 삭제할 column명
        """
        self.backup = self[column]
        
        for i in range(len(self[column])):
            self[column].iloc[i] = "" # 공백으로 값 변경
            
        return after
    
    def delete_row(self, rows):
        """
            delete_row : 행 삭제
            - rows : 삭제할 row명
        """
        self.backup = self.iloc[rows]
        del_df = self.drop(rows, axis=0)
        return del_df
    
    def delete_partial(self, column, num):
        """
            delete_partial : 부분 삭제
            - column : 부분 삭제할 column명
            - num : num번째 이후 값 삭제 = level로 지정
        """
        self.backup = self[column]
        splitData = self[column].str.split(' ') # 구분자를 띄어쓰기로 하여 split
        
        col = []
        for i in splitData:
            i = i[:num] # num번째 값까지만 남겨 놓음
            col.append(" ".join(i))
        self[column] = col
        after = self
        return after
    
    def masking(self,column,direction,number):           #(데이터프레임, 처리할 칼럼, 남겨둘 정보의 위치, 남겨둘 정보의 개수)
        self.backup = self[column]
        before_data=self.loc[:,column]                   

        after = self.astype({column : 'object'})           #columns 데이터 타입 변경

        for i in range(len(before_data.index)):
            string = str(before_data.iloc[i])
            length = len(string)                  
            m_length = length - number            #마스킹할 문자 길이 구하기

            if number > length:                                                           #남겨둘 문자의 길이가 전체 문자길이보다 작을 경우 
                string = string.replace(string[:],"*"*length)                             

            if direction == 'right':              #남겨둘 정보(마스킹하지 않을 정보)의 위치=오른쪽
                if number <= length:
                    string = string.replace(string[:-number],"*"*m_length)

            elif direction == 'left':             #남겨둘 정보(마스킹하지 않을 정보)의 위치=왼쪽
                if number <= length:
                    string = string.replace(string[number:],"*"*m_length)
            after.at[i,column] = string

        return after 
    
    def Drange(self,column,num):         #(데이터프레임, 적용할 칼럼, 수의 간격)
        self.backup = self[column]
        self = self.astype({column : 'int64'})        # float인 경우를 위해 int로 columns 데이터 타입 변경
        before_data = self.loc[:,column]
    
        d_min = before_data.min()         #데이터의 최소값 구하기
        ds_min = str(d_min)
        length = len(ds_min)-1
        ds_min = ds_min.replace(ds_min[1:],"0"*length)
        d_min = int(ds_min)
    
        d_max = before_data.max()         #데이터의 최대값 구하기
        ds_max = str(d_max)
        length = len(ds_max)-1
        re_max = str(int(ds_max[0])+1)          #ex)max= 60 -> 61은 결측치 -> 첫자리 1증가 60->70
        ds_max = ds_max.replace(ds_max[:],re_max+"0"*length)
        d_max = int(ds_max)+1
    
        bins = list(range(d_min, d_max, num))        #구간을 나누는 기준점 리스트

        after = self.astype({column : 'category'})
    
        re_cut = pd.cut(before_data, bins)
        self[column] = re_cut
        after = self
    
        return after 

    def Rounding(self,column,loc,job):  #데이터프레임, 컬럼, 위치, 작업        
        
        if job==0:#반올림            
            self[column] = round(self[column], loc)
            after=self
        elif job==1:#올림
            after[after.columns[column]] = math.ceil(after[after.columns[column]], loc)
        elif job==2:#내림
            after[after.columns[column]] = math.floor(after[after.columns[column]], loc)
        return after
    
    def Pretreatment(self,column): # count =1 인 값 제거
        list2=[]
        list3=self[column].tolist()
        for i in list3:
            if(list3.count(i)==1):
                i = ''
                list2.append(i)
            else:
                list2.append(i)
        list3=list2
        self[column]=list3
        after=self
         
    def object_normalize(self,column): #문자형 데이터 전처리 
        s = pd.Series(self[column]).value_counts()
        s = (s-s.mean())/s.std()
        l_overvalue =s[s<-3].index
        result = l_overvalue
        for i in result:
            after= self.loc[self[column]!=i]

In [4]:
# 전처리 클래스 
class pretreatment():    
    
    def value_normalize(after,column):   #수치형 데이터만 준비 완료 누를 때 이상치 제거 
            s = after[column]
            s = s[((s-s.mean())/s.std()>3) | ((s-s.mean())/s.std()<-3)]
            after = after[~after[column].isin(s)]

In [5]:
# 타입 변경 ui            
type_change_class = uic.loadUiType("type_change.ui")[0]
class TypeChangeWindow(QDialog, type_change_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.yes_button.clicked.connect(self.change)
        
    def change(self) :
        global type_to
        if self.radio_object.isChecked() == True :
            self.close()
            type_to = 'object'
        elif self.radio_int.isChecked() == True :
            self.close()
            type_to = 'int64'
        elif self.radio_float.isChecked() == True :
            self.close()
            type_to = 'float64'

In [6]:
# level 정하는 window
class LevelWindow(QWidget):
    col = ""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        uic.loadUi('levelWindow_widget.ui', self)

        self.label_col.setText(self.col)

In [6]:
# delimiter 고를 수 있는 level window
class LevelWindow_partial(QWidget):
    col = ""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        uic.loadUi('levelWindow_partial_widget.ui', self)
        
        self.label_col.setText(self.col)

In [7]:
# 비식별 적용 버튼 클릭 시, 새 창 띄워짐 (= anonymousWindow)
anonymous_class = uic.loadUiType("anonymousWindow.ui")[0]

class AnonymousWindow(QDialog, anonymous_class):
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.checked_method = [] # 기법 체크박스 값 리스트
        self.next_index = 2 # stackedWidget의 페이지 수 
        self.typelist = 0 # 있어 이런거..
        
        # 준식별자로 택한 column index
        global type_combo_res
        type_combo_res = np.array(type_combo_res) # list에서 해당 단어가 포함된 모든 index를 얻기 위해 numpy 사용
        self.QI_index = np.where(type_combo_res == '준식별자')[0] # 준식별자인 컬럼 index 리스트

        # 버튼 이벤트
        self.cancel_button.clicked.connect(self.closeWindow) # 취소 버튼 클릭 시, 창 닫힘
        self.next_button.clicked.connect(self.go_levelWindow) # next 버튼 클릭 시, addWidget한 위젯으로 이동할 수 있는 함수 호출
        
        # Window 이름 지정
        self.setWindowTitle("anonymous setting")
        
        # groupBox list 항목 -> horiziontal Layout 에 추가
        self.method_box_list = self.methodBox() # methodBox 함수를 이용해 groupbox 리스트를 받아옴
        for i in self.method_box_list :
            self.horizontalLayout_groupbox.addWidget(i) # groupbox list 하나씩 horizontalLayout에 addWidget 함
        
        self.show()
        
    def closeWindow(self):
        self.close()
    
    def methodBox(self): # 비식별 기법 체크박스 생성
        global data
        
        groupbox = [] # 준식별자에 적용할 checkbox의 groupbox를 담는 리스트
        j = 0 # groupbox index로 사용할 변수
        for i in self.QI_index:
            groupbox_objName = "groupBox" + str(i)
            groupbox.append(QGroupBox(data.columns[i] + " (준식별자)", objectName=groupbox_objName)) # groupbox 제목 지정해서 groupbox list에 담음
            
            method = ['열 삭제', '부분 삭제', '라운딩', '마스킹', '총계처리', '범주화'] # 비식별 기술 리스트
            chkbox_list = [] # 각 groupbox에 들어갈 checkbox 리스트

            check_idx = 0
            for item in method:
                checkbox_objName = "checkBox_" + str(i) + "_" + str(check_idx) 
                if(data[data.columns[i]].dtype == object ): #데이터 타입을 비교 
                    if(item =='부분 삭제' or item=='마스킹'):
                        item = item + '(추천)'
                    else:
                        pass
                else:
                    if(item == '라운딩' or item=='총계처리' or item == '범주화'):
                        item = item + '(추천)'
                    else:
                        pass
                chkbox_list.append(QCheckBox(item, objectName=checkbox_objName)) # 체크박스 리스트에 각 체크박스 항목을 넣는 for문
                check_idx += 1
            vbox = QVBoxLayout() 
            for chkbox in chkbox_list:
                vbox.addWidget(chkbox) # box layout에 체크박스들을 담은 리스트를 addWidget함
            groupbox[j].setLayout(vbox)
            groupbox[j].setLayout 
            j+=1 # groupbox index 증가
        return groupbox
    
    def go_levelWindow(self):
        global data    
        
        if self.typelist == 0:
            self.getTypeList() # getTypeList 함수는 한 번만 실행되게 !
            self.typelist += 1
        
        print("checked_method : ", self.checked_method)
        print("QI_index : ", self.QI_index)
        
        """ 
            - stackedWidget 추가하는 과정
             참고 ! index 0 : 기본 k 입력 받고 비식별 기법 적용하는 위젯
                    index 1 : 빈칸
                    index 2 : addWidget 한 내용
        """
        for method, qi in zip(self.checked_method, self.QI_index):
            if method.find('부분 삭제') != -1: # 부분삭제 비식별기법 사용 시,
                self.levelWindow_p = LevelWindow_partial()
                self.levelWindow_p.label_col.setText(data.columns[qi]) # 레벨트리 화면 제목 지정
                self.stackedWidget.addWidget(self.levelWindow_p)
                # LevelWindow_partial.col = data.columns[qi]
            else : # 그 외의 비식별기법 사용 시,
                self.levelWindow = LevelWindow()
                self.levelWindow.label_col.setText(data.columns[qi]) # 레벨트리 화면 제목 지정
                self.stackedWidget.addWidget(self.levelWindow)
                # LevelWindow.col = data.columns[qi]

        if self.next_index == 1+len(self.QI_index): # 마지막 stackedWidget일 때, 
            self.next_button.setText('완료') # 다음 버튼 대신 완료 버튼으로 변경
            self.next_button.clicked.connect(self.complete) # 완료 버튼 클릭 시, complete 함수 실행
                
        # stackedWidget index 이동
        print(self.next_index,"번째 stackedWidget")
        self.stackedWidget.setCurrentIndex(self.next_index)
        self.next_index += 1 # 다음버튼 클릭 후 그 다음에 다음버튼할 index
    
    def complete(self):
        self.close()
    
    def getTypeList(self): # 체크된 기법 리스트에 담는 과정     
        for i in self.QI_index: # method_box_list의 모든 그룹 박스
            for j in range(5): # 체크박스 (0~5 = range(6))
                checkbox_name = "checkBox_" + str(i) + "_" + str(j)
                chkbox = self.findChild(QCheckBox, checkbox_name)
                if chkbox.isChecked() : self.checked_method.append(chkbox.text()) 

In [9]:
form_class = uic.loadUiType("4thUI2.ui")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :    
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        
        # 로고 이미지
        qPixmapVar = QPixmap()
        qPixmapVar.load("../인터페이스/logo2.jpg")
        self.Logo.setPixmap(qPixmapVar)
        
        # 버튼 이벤트
        self.open_button.clicked.connect(self.open_file) # open_button 클릭 관련
        self.apply_button.clicked.connect(self.open_anonymousWindow) # '비식별 처리' 버튼 클릭 관련
        self.save_button.clicked.connect(self.save_file) # save_button 클릭 관련
        self.type_button.clicked.connect(self.type_change) # 타입 변경
        self.set_button.clicked.connect(self.set_after) # --
        
        self.loc = 0
        self.job = 0
        loc = self.loc
        
    def open_file(self) : # 파일 불러오기
        global data
        global count
        global info_header
        
        open_name = QFileDialog.getOpenFileName(self, 'Open File', './')       
        if (max(open_name) != ""):
            data = pd.read_csv(open_name[0], encoding='utf-8')
            self.status.setText(open_name[0])

            global after # 비식별 처리 후 데이터를 저장할 전역변수 after
            after = data
            count = list()
            
            # 개인정보 유형(식별자, 준식별자, 민감정보, 비민감정보) Combo Box
            global type_combo_list # 개인정보 유형 콤보박스들을 담는 리스트
            type_combo_list = []
            for i in range(len(data.columns)):
                type_combo = QComboBox()
                type_combo.addItems(['비민감정보','식별자', '준식별자', '민감정보'])
                type_combo_list.append(type_combo)
            
            # row header 추가하기 위한 list 생성
            row = [*range(1, preview + 2)]
            row.insert(0,"타입")
            row.insert(0,"")
            row = list(map(str, row))
            
            # table_1 widget 행, 열 추가
            self.table_1.setRowCount(preview + 2) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
            self.table_1.setColumnCount(preview + 1)
            self.table_1.setHorizontalHeaderLabels(list(data))
            self.table_1.setVerticalHeaderLabels(row)

            # table_1 widget에 개인정보 유형 콤보박스 추가
            for i in range(len(data.columns)):
                self.table_1.setCellWidget(0, i, type_combo_list[i]) # 0번째 행에 개인정보 유형 Combo Box
            
            # 비식별 전 table_1에 
            # table_3 widget에 info추가 하기 위해서 수치형 개수 count
            for i in range(len(data.columns)) :
                if data[data.columns[i]].dtype == 'object' :
                    self.table_1.setItem(1, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'int64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'float64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
                
                if (data[data.columns[i]].dtype == 'int64') | (data[data.columns[i]].dtype == 'float64') :
                    count.append(i)

            # table_1 widget에 불러온 파일의 데이터 추가
            for i in range (preview) :
                for j in range (len(list(data))) :
                    self.table_1.setItem(i + 2, j, QTableWidgetItem(format(data.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력
            
            # 수치형 데이터 table_3에 min, max ,median 등
            self.table_3.setRowCount(len(count))
            self.table_3.setColumnCount(len(list(info_header)))
            self.table_3.setHorizontalHeaderLabels(list(info_header))
            
            # 수치형 데이터의 이름, 최솟값, 최댓값, 평균, 중앙값, 표준편차, 최빈값 출력
            for i in range(len(count)) :
                self.table_3.setItem(i, 0, QTableWidgetItem(format(data.columns[count[i]])))
                self.table_3.setItem(i, 1, QTableWidgetItem(format(data[data.columns[count[i]]].min())))
                self.table_3.setItem(i, 2, QTableWidgetItem(format(data[data.columns[count[i]]].max())))
                self.table_3.setItem(i, 3, QTableWidgetItem("%.2lf"%(data[data.columns[count[i]]].mean())))
                self.table_3.setItem(i, 4, QTableWidgetItem(format(data[data.columns[count[i]]].median())))
                self.table_3.setItem(i, 5, QTableWidgetItem("%.3lf"%(data[data.columns[count[i]]].std())))
                self.table_3.setItem(i, 6, QTableWidgetItem(format(data[data.columns[count[i]]].value_counts().index[0])))
        else:
            print('파일 선택 안함')
            
    # 타입 변경
    def type_change(self) :
        global type_to
        global data
        
        type_change_Window = TypeChangeWindow()
        type_change_Window.exec_()
        column = self.table_1.currentColumn()
        data[data.columns[column]] = data[data.columns[column]].astype(type_to)
        for i in range(len(data.columns)) :
                if data[data.columns[i]].dtype == 'object' :
                    self.table_1.setItem(1, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'int64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif data[data.columns[i]].dtype == 'float64' :
                    self.table_1.setItem(1, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
    
    def set_after(self) :
        global type_combo_list
        global type_combo_res
        global count
        global info_header
        
        type_combo_res = [] # 개인정보 유형 콤보박스 선택 값 넣는 리스트
        for combo in type_combo_list:
            type_combo_res.append(combo.currentText()) # 현재 콤보박스 text를 type_combo_res에 append

        # 수치형 전처리 
        for i in range(len(after.columns)):
            if type_combo_res[i] == '식별자' :
                after[after.columns[i]] = ''
            if(after[after.columns[i]].dtype != object):
                pretreatment.value_normalize(after, after.columns[i])
            else:
                pass
            

        # row header 추가하기 위한 list 생성
        row = [*range(1, preview + 1)]
        row.insert(0,"타입")
        row = list(map(str, row))
        
        self.table_2.setRowCount(preview + 1) # data type, 개인정보 유형 -> "2행" 이후 데이터 출력
        self.table_2.setColumnCount(len(list(after)))
        self.table_2.setHorizontalHeaderLabels(list(after))
        self.table_2.setVerticalHeaderLabels(row)
        
        # table_2 widget에 불러온 파일의 데이터 추가
        for i in range (preview) :
                for j in range (len(list(after))) :
                    self.table_2.setItem(i + 1, j, QTableWidgetItem(format(after.values.tolist()[i][j]))) # 개인정보 유형, 데이터타입 -> "2행" 이후 데이터 출력
        
        # table_2 widget에 데이터 타입 추가
        for i in range(len(after.columns)) :
                if after[after.columns[i]].dtype == 'object' :
                    self.table_2.setItem(0, i, QTableWidgetItem('문자열')) # 1번째 행에 데이터 타입
                elif after[after.columns[i]].dtype == 'int64' :
                    self.table_2.setItem(0, i, QTableWidgetItem('정수')) # 1번째 행에 데이터 타입
                elif after[after.columns[i]].dtype == 'float64' :
                    self.table_2.setItem(0, i, QTableWidgetItem('실수')) # 1번째 행에 데이터 타입
        
        # 수치형 데이터 table_3에 min, max ,median 등
        self.table_4.setRowCount(len(count))
        self.table_4.setColumnCount(len(list(info_header)))
        self.table_4.setHorizontalHeaderLabels(list(info_header))

        # 수치형 데이터의 이름, 최솟값, 최댓값, 평균, 중앙값, 표준편차, 최빈값 출력
        for i in range(len(count)) :
            self.table_4.setItem(i, 0, QTableWidgetItem(format(after.columns[count[i]])))
            self.table_4.setItem(i, 1, QTableWidgetItem(format(after[after.columns[count[i]]].min())))
            self.table_4.setItem(i, 2, QTableWidgetItem(format(after[after.columns[count[i]]].max())))
            self.table_4.setItem(i, 3, QTableWidgetItem("%.2lf"%(after[after.columns[count[i]]].mean())))
            self.table_4.setItem(i, 4, QTableWidgetItem(format(after[after.columns[count[i]]].median())))
            self.table_4.setItem(i, 5, QTableWidgetItem("%.3lf"%(after[after.columns[count[i]]].std())))
            self.table_4.setItem(i, 6, QTableWidgetItem(format(after[after.columns[count[i]]].value_counts().index[0])))

    def open_anonymousWindow(self):
        # 준식별자 값 전달 필요
        anonymousWindow = AnonymousWindow()
        anonymousWindow.exec_()
        anonymousWindow.show()
                
    def save_file(self) : # 비식별처리 후 파일 저장
        save_name = QFileDialog.getSaveFileName(self, 'Open File', './')
        after.to_csv(save_name[0], encoding='utf-8', index=False)
        
if __name__ == "__main__" :
    app = QApplication(sys.argv) 
    myWindow = WindowClass()
    myWindow.show()
    app.exec_()

go_levelWindow
checked_method :  ['부분 삭제(추천)', '열 삭제', '부분 삭제(추천)']
QI_index :  [0 1 2]
2 번째 stackedWidget
go_levelWindow
checked_method :  ['부분 삭제(추천)', '열 삭제', '부분 삭제(추천)']
QI_index :  [0 1 2]
3 번째 stackedWidget
go_levelWindow
checked_method :  ['부분 삭제(추천)', '열 삭제', '부분 삭제(추천)']
QI_index :  [0 1 2]
4 번째 stackedWidget
